In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.linear_model import Lasso
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor, RadiusNeighborsClassifier
from sklearn.metrics import roc_auc_score
from sklearn.multioutput import ClassifierChain


In [ ]:
training_df = pd.read_csv('drive/MyDrive/Colab Notebooks/Data/training_set_features.csv', index_col=0)
training_df.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,55 - 64 Years,< 12 Years,White,Female,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,35 - 44 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,3.0,1.0,1.0,4.0,1.0,2.0,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,3.0,3.0,5.0,5.0,4.0,1.0,65+ Years,12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,3.0,2.0,3.0,1.0,4.0,45 - 54 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [ ]:
training_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Data columns (total 35 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   h1n1_concern                 26615 non-null  float64
 1   h1n1_knowledge               26591 non-null  float64
 2   behavioral_antiviral_meds    26636 non-null  float64
 3   behavioral_avoidance         26499 non-null  float64
 4   behavioral_face_mask         26688 non-null  float64
 5   behavioral_wash_hands        26665 non-null  float64
 6   behavioral_large_gatherings  26620 non-null  float64
 7   behavioral_outside_home      26625 non-null  float64
 8   behavioral_touch_face        26579 non-null  float64
 9   doctor_recc_h1n1             24547 non-null  float64
 10  doctor_recc_seasonal         24547 non-null  float64
 11  chronic_med_condition        25736 non-null  float64
 12  child_under_6_months         25887 non-null  float64
 13  health_worker   

In [ ]:
training_df.describe()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children
count,26615.000000,26591.000000,26636.000000,26499.000000,26688.000000,26665.000000,26620.00000,26625.000000,26579.000000,24547.000000,24547.000000,25736.000000,25887.000000,25903.000000,14433.00000,26316.000000,26319.000000,26312.000000,26245.000000,26193.000000,26170.000000,26458.000000,26458.000000
mean,1.618486,1.262532,0.048844,0.725612,0.068982,0.825614,0.35864,0.337315,0.677264,0.220312,0.329735,0.283261,0.082590,0.111918,0.87972,3.850623,2.342566,2.357670,4.025986,2.719162,2.118112,0.886499,0.534583
std,0.910311,0.618149,0.215545,0.446214,0.253429,0.379448,0.47961,0.472802,0.467531,0.414466,0.470126,0.450591,0.275266,0.315271,0.32530,1.007436,1.285539,1.362766,1.086565,1.385055,1.332950,0.753422,0.928173
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000
25%,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,3.000000,1.000000,1.000000,4.000000,2.000000,1.000000,0.000000,0.000000
50%,2.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.00000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,4.000000,2.000000,2.000000,4.000000,2.000000,2.000000,1.000000,0.000000
75%,2.000000,2.000000,0.000000,1.000000,0.000000,1.000000,1.00000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.00000,5.000000,4.000000,4.000000,5.000000,4.000000,4.000000,1.000000,1.000000
max,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,3.000000,3.000000


In [ ]:
LABELS = training_df.columns[training_df.dtypes == "object"].values
NUMERIC = training_df.columns[training_df.dtypes != "object"].values
print(LABELS)
print(NUMERIC)

['age_group' 'education' 'race' 'sex' 'income_poverty' 'marital_status'
 'rent_or_own' 'employment_status' 'hhs_geo_region' 'census_msa'
 'employment_industry' 'employment_occupation']
['h1n1_concern' 'h1n1_knowledge' 'behavioral_antiviral_meds'
 'behavioral_avoidance' 'behavioral_face_mask' 'behavioral_wash_hands'
 'behavioral_large_gatherings' 'behavioral_outside_home'
 'behavioral_touch_face' 'doctor_recc_h1n1' 'doctor_recc_seasonal'
 'chronic_med_condition' 'child_under_6_months' 'health_worker'
 'health_insurance' 'opinion_h1n1_vacc_effective' 'opinion_h1n1_risk'
 'opinion_h1n1_sick_from_vacc' 'opinion_seas_vacc_effective'
 'opinion_seas_risk' 'opinion_seas_sick_from_vacc' 'household_adults'
 'household_children']


In [ ]:
training_df = pd.get_dummies(training_df, dummy_na=True, dtype='int64')
training_df.info()
training_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26707 entries, 0 to 26706
Columns: 117 entries, h1n1_concern to employment_occupation_nan
dtypes: float64(23), int64(94)
memory usage: 24.0 MB


,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,age_group_nan,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,education_nan,race_Black,race_Hispanic,race_Other or Multiple,race_White,race_nan,sex_Female,...,employment_industry_ldnlellj,employment_industry_mcubkhph,employment_industry_mfikgejo,employment_industry_msuufmds,employment_industry_nduyfdeo,employment_industry_phxvnwax,employment_industry_pxcmvdjn,employment_industry_qnlwzans,employment_industry_rucpziij,employment_industry_saaquncn,employment_industry_vjjrobsf,employment_industry_wlfvacwt,employment_industry_wxleyezf,employment_industry_xicduogh,employment_industry_xqicxuve,employment_industry_nan,employment_occupation_bxpfxfdn,employment_occupation_ccgxvspp,employment_occupation_cmhcxjea,employment_occupation_dcjcmpih,employment_occupation_dlvbwzss,employment_occupation_emcorrxb,employment_occupation_haliazsg,employment_occupation_hfxkjkmi,employment_occupation_hodpvpew,employment_occupation_kldqjyjy,employment_occupation_mxkfnird,employment_occupation_oijqvulv,employment_occupation_pvmttkik,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv,employment_occupation_nan
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,2.0,2.0,1.0,2.0,0.0,0.0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,4.0,4.0,4.0,2.0,4.0,0.0,0.0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,0.0,0.0,NaN,3.0,1.0,1.0,4.0,1.0,2.0,2.0,0.0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,3.0,3.0,5.0,5.0,4.0,1.0,0.0,0.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,3.0,2.0,3.0,1.0,4.0,1.0,0.0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# training_df.fillna(-1000, inplace=True)
# training_df.head()

In [ ]:
# change_type = lambda x: x.astype('int64')
# training_df = training_df.apply(change_type, axis=0)
# training_df.head()

In [ ]:
training_labels = pd.read_csv('drive/MyDrive/Colab Notebooks/Data/training_set_labels.csv', index_col=0)
change_type = lambda x: x.astype('int64')
training_labels = training_labels.apply(change_type, axis=0)
training_labels.head()

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
0,0,0
1,0,1
2,0,0
3,0,1
4,0,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(training_df, training_labels, test_size=0.2, random_state=42)
y_train

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
24706,0,0
5393,0,1
20898,1,1
3429,0,0
8731,0,0
...,...,...
21575,0,1
5390,0,0
860,0,0


In [ ]:
clf_knn = ClassifierChain(KNeighborsClassifier(), order=[1,0])
pl_knn = Pipeline([
               ('scaler', MinMaxScaler()),
               ('imputer', SimpleImputer(strategy='constant',fill_value=-1000)),
               ('clf_knn', clf_knn)
])

In [ ]:
clf_rnn = ClassifierChain(RadiusNeighborsClassifier(), order=[1,0])
pl_rnn = Pipeline([
               ('scaler', MinMaxScaler()),
               ('imputer', SimpleImputer(strategy='constant',fill_value=-1000)),
               ('clf_knn', clf_rnn)
])

In [ ]:
# lr = LogisticRegression(max_iter=10000)
# clf_lr = ClassifierChain(lr, order=[1,0])
# pl_lr = Pipeline([
#                ('scaler', MinMaxScaler()),
#                ('imputer', SimpleImputer(strategy='constant',fill_value=-1000)),
#                ('clf_lr', clf_lr)
# ])

In [ ]:
## Haven't gotten good scores with linear SVC, trying SVC

svc = SVC(max_iter=20000, C=.1, kernel='linear', probability=True)
clf_svc = ClassifierChain(svc, order=[1,0])
pl_svc = Pipeline([
               ('scaler', MinMaxScaler()),
               ('imputer', SimpleImputer(strategy='most_frequent')),
               ('clf_svc', clf_svc)
])

# Best: {'clf_svc__base_estimator__C': 0.1, 'clf_svc__base_estimator__kernel': 'linear'}
# 0.7784953856454875

In [ ]:
# clf = ClassifierChain(order=[1,0])
# pl = Pipeline([
#                ('scaler', MinMaxScaler()),
#                ('imputer', SimpleImputer(strategy='constant',fill_value=-1000)),
#                ('clf', clf)
# ])

TypeError: ignored

In [ ]:
model_params = {
    'svc': {
        'model': pl_svc,
        'params': {
            'clf_svc__base_estimator__C': [.1],
            'clf_svc__base_estimator__kernel': ['linear']
        }
    },
    # 'knn': {
    #     'model': pl_knn,
    #     'params': {
    #         'clf_knn__base_estimator__n_neighbors': np.arange(1,10,1),
    #         'clf_knn__base_estimator__leaf_size': np.arange(20,40,1),
    #         'clf_knn__base_estimator__p': (1,2),
    #         'clf_knn__base_estimator__weights': ('uniform', 'distance'),
    #         'clf_knn__base_estimator__metric': ('minkowski', 'chebyshev')
    #     }
    # },
    # 'rnn': {
    #     'model': pl_rnn,
    #     'params': {
    #         'clf_rnn__base_estimator__n_neighbors': np.arange(1,10,1),
    #         'clf_rnn__base_estimator__leaf_size': np.arange(20,40,1),
    #         'clf_rnn__base_estimator__p': (1,2),
    #         'clf_rnn__base_estimator__weights': ('uniform', 'distance'),
    #         'clf_rnn__base_estimator__metric': ('minkowski', 'chebyshev')
    #     }
    # }
}

In [ ]:
scores = []
for model_name, mp in model_params.items():
  model_cv = GridSearchCV(mp['model'], mp['params'], cv=3, return_train_score=False, scoring='roc_auc')
  model_cv.fit(X_train[important_features.columns], y_train)
  scores.append({
      'model': model_name,
      'best_score': model_cv.best_score_,
      'best_params': model_cv.best_params_
  })

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/s

In [ ]:
score_df = pd.DataFrame(scores,columns=['model', 'best_score', 'best_params'])
print(score_df['best_params'][0])
print(score_df['best_score'][0])

{'clf_svc__base_estimator__C': 0.1, 'clf_svc__base_estimator__kernel': 'linear'}
0.7784953856454875


In [ ]:
# # Maybe try reversing the chain next?

# model_cv = GridSearchCV(pl, param_grid=parameters, cv=5, scoring='roc_auc')
# model_cv.fit(X_train, y_train)
# roc = model_cv.score(X_test, y_test)
# print("Tuned Parameters: {}".format(model_cv.best_params_))
# print("Tuned roc_auc: {}".format(roc))

Tuned Parameters: {'clf__order': [1, 0]}
Tuned roc_auc: 0.7910982774212607


In [ ]:

# clf.fit(X_train, y_train)
# print("Accuracy: {}".format(clf.score(X_test, y_test)))

In [ ]:
# clf1 = OneVsRestClassifier(LinearSVC(max_iter=5000))
# clf1.fit(X_train, y_train)
# print("Accuracy: {}".format(clf1.score(X_test, y_test)))

In [ ]:
# This code gets important features from the dataset

lasso = Lasso(alpha=0.5)
pl_lasso = Pipeline([
               ('scaler', MinMaxScaler()),
               ('imputer', SimpleImputer(strategy='constant',fill_value=-1000)),
               ('clf_lasso', lasso)
])
pl_lasso.fit(X_train, y_train)
feature_coef = pd.DataFrame(pl_lasso['clf_lasso'].coef_, columns=training_df.columns)
important_features = feature_coef.loc[:, (feature_coef != 0.0).any()]
important_features.columns

Index(['behavioral_antiviral_meds', 'behavioral_touch_face',
       'doctor_recc_h1n1', 'doctor_recc_seasonal', 'chronic_med_condition',
       'child_under_6_months', 'health_insurance', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_risk', 'household_adults'],
      dtype='object')

In [ ]:
# pl = Pipeline([
#                ('scaler', MinMaxScaler()),
#                ('linear_svc', LinearSVC())
#                ])



In [ ]:
# from sklearn.multiclass import OneVsRestClassifier


# linear_svc = OneVsRestClassifier(LinearSVC(max_iter=10000))
# linear_svc.fit(X_test, y_test)

In [ ]:
# model_cv.get_params().keys()

In [ ]:
pl_svc.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
         steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='most_frequent',
                               verbose=0)),
                ('clf_svc',
                 ClassifierChain(base_estimator=SVC(C=0.1, break_ties=False,
                                                    cache_size=200,
                                                    class_weight=None,
                                                    coef0=0.0,
                                                    decision_function_shape='ovr',
                                                    degree=3, gamma='scale',
                                                    kernel='linear',
                                                    max_iter=20000,
                                                    probability=True,


In [ ]:
accuracy = pl_svc.score(X_test, y_test)
print("\nAccuracy: ", accuracy)

# Best so far - Accuracy:  0.6323474354174466


Accuracy:  0.6785847997004867


In [ ]:
y_preds = pl_svc.predict_proba(X_test)

In [ ]:
y_preds

array([[0.05183661, 0.23453143],
       [0.04230436, 0.18361675],
       [0.06264633, 0.86790197],
       ...,
       [0.8354531 , 0.53411424],
       [0.10769209, 0.59162735],
       [0.07869112, 0.67737367]])

In [ ]:
roc_auc_score(y_test, y_preds)
# knn best: 0.8306339018164306
# svc best: 0.8383080315800618

0.8383080315800618

In [ ]:
# Retrain on entire dataset
pl_svc.fit(training_df, training_labels)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=20000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
         steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))),
                ('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='most_frequent',
                               verbose=0)),
                ('clf_svc',
                 ClassifierChain(base_estimator=SVC(C=0.1, break_ties=False,
                                                    cache_size=200,
                                                    class_weight=None,
                                                    coef0=0.0,
                                                    decision_function_shape='ovr',
                                                    degree=3, gamma='scale',
                                                    kernel='linear',
                                                    max_iter=20000,
                                                    probability=True,


In [ ]:
test_labels = pd.read_csv('drive/MyDrive/Colab Notebooks/Data/test_set_features.csv', index_col=0)
test_labels.head()

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,1.0,5.0,1.0,1.0,35 - 44 Years,College Graduate,Hispanic,Female,"> $75,000",Not Married,Rent,Employed,mlyzmhmf,"MSA, Not Principle City",1.0,0.0,atmlpfrs,hfxkjkmi
26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,4.0,1.0,1.0,18 - 34 Years,12 Years,White,Male,Below Poverty,Not Married,Rent,Employed,bhuqouqj,Non-MSA,3.0,0.0,atmlpfrs,xqwwgdyp
26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0,4.0,2.0,5.0,4.0,4.0,55 - 64 Years,College Graduate,White,Male,"> $75,000",Married,Own,Employed,lrircsnp,Non-MSA,1.0,0.0,nduyfdeo,pvmttkik
26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,4.0,2.0,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Married,Own,Not in Labor Force,lrircsnp,"MSA, Not Principle City",1.0,0.0,NaN,NaN
26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,2.0,4.0,4.0,4.0,2.0,35 - 44 Years,12 Years,Black,Female,"<= $75,000, Above Poverty",Not Married,Own,Employed,lzgpxyit,Non-MSA,0.0,1.0,fcxhlnwr,mxkfnird


In [ ]:
test_labels = pd.get_dummies(test_labels, dummy_na=True, dtype='int64')
test_labels.info()
test_labels.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26708 entries, 26707 to 53414
Columns: 117 entries, h1n1_concern to employment_occupation_nan
dtypes: float64(23), int64(94)
memory usage: 24.0 MB


,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,household_adults,household_children,age_group_18 - 34 Years,age_group_35 - 44 Years,age_group_45 - 54 Years,age_group_55 - 64 Years,age_group_65+ Years,age_group_nan,education_12 Years,education_< 12 Years,education_College Graduate,education_Some College,education_nan,race_Black,race_Hispanic,race_Other or Multiple,race_White,race_nan,sex_Female,...,employment_industry_ldnlellj,employment_industry_mcubkhph,employment_industry_mfikgejo,employment_industry_msuufmds,employment_industry_nduyfdeo,employment_industry_phxvnwax,employment_industry_pxcmvdjn,employment_industry_qnlwzans,employment_industry_rucpziij,employment_industry_saaquncn,employment_industry_vjjrobsf,employment_industry_wlfvacwt,employment_industry_wxleyezf,employment_industry_xicduogh,employment_industry_xqicxuve,employment_industry_nan,employment_occupation_bxpfxfdn,employment_occupation_ccgxvspp,employment_occupation_cmhcxjea,employment_occupation_dcjcmpih,employment_occupation_dlvbwzss,employment_occupation_emcorrxb,employment_occupation_haliazsg,employment_occupation_hfxkjkmi,employment_occupation_hodpvpew,employment_occupation_kldqjyjy,employment_occupation_mxkfnird,employment_occupation_oijqvulv,employment_occupation_pvmttkik,employment_occupation_qxajmpny,employment_occupation_rcertsgn,employment_occupation_tfqavkke,employment_occupation_ukymxvdu,employment_occupation_uqqtjvyb,employment_occupation_vlluhbov,employment_occupation_xgwztkwe,employment_occupation_xqwwgdyp,employment_occupation_xtkaffoo,employment_occupation_xzmlyyjv,employment_occupation_nan
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
26707,2.0,2.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,1.0,1.0,5.0,1.0,1.0,1.0,0.0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
26708,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,1.0,1.0,4.0,1.0,1.0,3.0,0.0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
26709,2.0,2.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,5.0,4.0,2.0,5.0,4.0,4.0,1.0,0.0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
26710,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,4.0,2.0,2.0,4.0,4.0,2.0,1.0,0.0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
26711,3.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,5.0,2.0,4.0,4.0,4.0,2.0,0.0,1.0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
test_features_df = pl_svc.predict_proba(test_labels)

In [ ]:
test_features_df

array([[0.02365157, 0.26399452],
       [0.05276276, 0.04858957],
       [0.5202906 , 0.58759966],
       ...,
       [0.09417663, 0.19556713],
       [0.02532268, 0.36229921],
       [0.3289678 , 0.45555233]])

In [ ]:
submission_df = pd.read_csv('drive/MyDrive/Colab Notebooks/Data/submission_format.csv', 
                            index_col="respondent_id")
submission_df.head()

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.5,0.7
26708,0.5,0.7
26709,0.5,0.7
26710,0.5,0.7
26711,0.5,0.7


In [ ]:
submission_df['h1n1_vaccine'] = test_features_df[:,0]
submission_df['seasonal_vaccine'] = test_features_df[:,1]
submission_df.head()

,h1n1_vaccine,seasonal_vaccine
respondent_id,,
26707,0.023652,0.263995
26708,0.052763,0.048590
26709,0.520291,0.587600
26710,0.616042,0.882879
26711,0.052136,0.483100


In [ ]:
submission_df.to_csv('drive/MyDrive/Colab Notebooks/Data/my_submission.csv', index=True)
!head 'drive/MyDrive/Colab Notebooks/Data/my_submission.csv'

respondent_id,h1n1_vaccine,seasonal_vaccine
26707,0.023651566244982,0.2639945237506362
26708,0.052762755514321545,0.048589572727936146
26709,0.520290597147098,0.5875996624041866
26710,0.6160416136306949,0.8828785783732612
26711,0.05213619683988799,0.4830995223511114
26712,0.4568132073122752,0.9233408697161349
26713,0.5531793356373947,0.5806379874563607
26714,0.0752651421251384,0.20097927740352028
26715,0.026157538351606044,0.1532141066946099
